This script automates the setup for a large batch of polymer gel simulations defined by parameters in `gel_configs`.

It iterates through run IDs, charge, volume fraction, crosslinking percentage, and agent force field. After verifying the existence of required input paths, the script prints the complete `pypresso` command lines necessary to launch these production runs, including specific equilibration and configuration details.

In [ ]:
import sys

sys.path.append("../configs")  # to include gel_configs.py
from gel_configs import gel_configs

import itertools
from pathlib import Path

import numpy as np

{'charge_per_bead': [0.05, 0.15, 0.25],
 'volume_fraction': [0.3, 0.45, 0.6],
 'crosslink_percentage': [0.2, 0.7],
 'agent_force_field': ['none', 'harmonic', 'exp']}

In [ ]:
rng = np.random.default_rng(874356)

gel_dir_root = Path("../data/gel_dirs/production_runs")


In [ ]:
rng_base_dirs = np.random.default_rng(123)
rng_seeds = np.random.default_rng(12345)

run_ids = range(5)

# print commands for running
for (
    charge_per_bead,
    run_id,
    volume_fraction,
    crosslink_percentage,
    agent_force_field,
) in itertools.product(
    gel_configs["charge_per_bead"],
    run_ids,
    gel_configs["volume_fraction"],
    gel_configs["crosslink_percentage"],
    gel_configs["agent_force_field"],
):
    gel_path = (
        gel_dir_root
        / f"run={run_id}_ff={agent_force_field}_charge={charge_per_bead}_vf={volume_fraction}_crosslink={crosslink_percentage}"
    )

    assert gel_path.exists()

    # when no force field is used we do not need to "diffuse" (equilibrate) the agents first so we can save computation time (but better set to 1 instead of zero to avoid logic edge-cases in various analysis scripts that expect some agent diffusion steps)
    initial_diff_steps = 1 if agent_force_field == "none" else 10**4

    command = (
        "pypresso ./src/main.py"
        f" {gel_path.resolve()}"
        " --steps_melt_eq_max 1000"
        " --bond_type fene"
        " --n_chains 70"
        " --chain_length 60"
        " --n_agents_per_crosslink 2"
        " --continue_past_crosslinking 1"
        " --steps_gel_eq_max 100000"
        f" --initial_diff_steps {initial_diff_steps}"
        # f" --seed {rng_seeds.integers(10**7, 10**8 - 1)}"
        f" --charge_per_bead {charge_per_bead}"
        f" --volume_fraction {volume_fraction}"
        f" --agent_force_field {agent_force_field}"
        f" --crosslink_percentage {crosslink_percentage}"
    )
    print(command)

pypresso ./src/main.py /Users/r/Documents/work/icp/microgel/code/data/raw/30112024_forcefield_eq_time_fixed/run=0_ff=none_charge=0.05_vf=0.3_crosslink=0.2 --steps_melt_eq_max 1000 --bond_type fene --n_chains 70 --chain_length 60 --n_agents_per_crosslink 2 --continue_past_crosslinking 1 --steps_gel_eq_max 100000 --initial_diff_steps 1 --charge_per_bead 0.05 --volume_fraction 0.3 --agent_force_field none --crosslink_percentage 0.2
pypresso ./src/main.py /Users/r/Documents/work/icp/microgel/code/data/raw/30112024_forcefield_eq_time_fixed/run=0_ff=harmonic_charge=0.05_vf=0.3_crosslink=0.2 --steps_melt_eq_max 1000 --bond_type fene --n_chains 70 --chain_length 60 --n_agents_per_crosslink 2 --continue_past_crosslinking 1 --steps_gel_eq_max 100000 --initial_diff_steps 10000 --charge_per_bead 0.05 --volume_fraction 0.3 --agent_force_field harmonic --crosslink_percentage 0.2
pypresso ./src/main.py /Users/r/Documents/work/icp/microgel/code/data/raw/30112024_forcefield_eq_time_fixed/run=0_ff=exp_c